HACETTEPE ÜNİVERSİTESİ
YAZILIM MÜHENDİSLİĞİ YÜKSEK LİSANS PROGRAMI
DÖNEM PROJESİ BYZ695
N21239069 - SERCAN YILDIRIM
DOÇ.DR.AYDIN KAYA

In [ ]:
#!pip install selenium
#!pip install webdrivermanager
#!pip install webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
# ChromeDriver yolu
chrome_driver_path = "C:\\Users\Yldrm\Desktop\cromedriver\chromedriver.exe"  

# Chrome seçenekleri
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Tarayıcıyı tam ekran başlat

# ChromeDriver servisini başlatma
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)


In [ ]:
# URL'ye erişim
url = "https://www.emlakjet.com/satilik-daire/ankara/49/"
driver.get(url)

chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

# Sayfanın yüklenmesi için kısa bir bekleme süresi
driver.implicitly_wait(10)  # Sayfanın tamamen yüklenmesi için 10 saniye bekleme

print(driver.title)


In [ ]:
def listToString(s):  
    str1 = " " 
    return (str1.join(s)) 

In [ ]:
driver.set_page_load_timeout(60)
# Time-out hatalarını yönetebilmek için süresi uzatıldı

In [ ]:
# İlan Sayfalarını dolaşabilmek için döngü yapıldı
sf = 1 
while sf <= 50:
    # Sayfa içindeki ilanları dolaşabilmek için döngü yapıldı.
    # ilk 9 sf 39 / 10 dan itibaren 35
    il = 1
    while il <= 35:
        # Tıklanması istenmeyen reklamların indisleri liste yapıldı.
        atlama_listesi = [3 , 8 , 9 , 11 , 14 , 18 , 25 , 26 , 33 , 40 ]
        u = 0
        while u < len(atlama_listesi):
            if atlama_listesi[u] == il:
                il = il+1
                u=0
            else:
                u = u + 1 
        
        # Tıklanacak konumda üst üste açılan pencere-pop-up lar olduğunda
        # hataya düşmemesi için eklendi. 
        try:
            driver.implicitly_wait(20)
            # xpath = f"//*[@id='listing-search-wrapper']/div[{il}]"
            xpath = "//*[@id='listing-search-wrapper']/div["+str(il)+"]"
            tikla = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, xpath))                              
             )
            # tikla = driver.find_element(By.XPATH, "//*[@id='listing-search-wrapper']/div["+str(il)+"]")
            driver.execute_script("arguments[0].scrollIntoView(true);", tikla)
            driver.execute_script("window.scrollBy(0, -100);") 
            driver.implicitly_wait(20)
            tikla.click()                                                               
            driver.implicitly_wait(10)
        except ElementClickInterceptedException:
            driver.implicitly_wait(20)
            # overlapping_element xpath //*[@id="listing-search-wrapper"]/div[1]/div
            overlapping_element = driver.find_element(By.XPATH, "//*[@id='listing-search-wrapper']/div["+ str(il)+"]/div")
            driver.execute_script("arguments[0].style.visibility='hidden'", overlapping_element)
            driver.execute_script("arguments[0].scrollIntoView(true);", tikla)
            driver.execute_script("window.scrollBy(0, -150);")
            print( str(sf) + " - " + str(il) + " excepte düştü.")
            tikla.click()
            driver.implicitly_wait(10)
            driver.execute_script("arguments[0].style.visibility='visible'", overlapping_element)
              
        driver.implicitly_wait(10)
        elements = driver.find_elements(By.CSS_SELECTOR, "._3tH_Nw") 
        driver.implicitly_wait(10)
        fiyatlar = driver.find_elements(By.CSS_SELECTOR,"._2TxNQv")
        detaylar = []
        fiyat = []
        for i in fiyatlar:
            fiyat.append(i.text)
        for k in elements:
            detaylar.append(k.text)
        det_str = listToString(detaylar)
        ayrı = det_str.split("\n")
        df = pd.DataFrame(ayrı)
        df_yeni = df[:]
        df_yeni = df_yeni.reset_index()
        df_yeni.drop("index", axis = 1, inplace = True)
        df_liste = df_yeni.values.tolist()
        içerikler =[]
        l = 1
        while l <= 27:
            içerikler.append(df_liste[l])
            l = l+2
        fiyat_sade = fiyat[1].strip()
        fiyat_sade = fiyat_sade.replace("TL","")
        içerikler.append([fiyat_sade])
        df_içerikler = pd.DataFrame(içerikler).T
        df_içerikler.to_csv(r"emlakjetveri.csv",encoding="utf-8",index=False, mode="a",header=False)
        il = il+1
        driver.execute_script("window.history.go(-1)")
        driver.implicitly_wait(10)         
    
    driver.implicitly_wait(10)
    sf = sf + 1
    print(str(sf-1) +  "  . sayfa tamamlandı")
    driver.get("https://www.emlakjet.com/satilik-daire/ankara/"+str(sf)+"/")
    driver.implicitly_wait(10)


In [ ]:
print(str(sf-1) + "Veri Toplama İşlemi Tamamlandı")